In [6]:
import pandas as pd

import boto3
import numpy as np
import io
# import awswrangler as wr
import sagemaker
from sagemaker import get_execution_role
from sagemaker.session import s3_input, Session
import pickle
from sagemaker.amazon.amazon_estimator import get_image_uri

from sklearn.preprocessing import OneHotEncoder
# Save the encoded categorical model
import tempfile
import boto3
import joblib

import s3fs
from sagemaker.predictor import csv_serializer
import tempfile
import joblib
from sagemaker.predictor import csv_serializer

import warnings
warnings.filterwarnings('ignore')

In [7]:
sess = sagemaker.Session()
role = get_execution_role()

bucket = "humana-data"
prefix_rawdata = 'rawdata/original_raw_files'
prefix_rawdata_train = 'rawdata/original_raw_files/train'
# prefix_fe_data = "intermediate/data"
# prefix_metadata = "intermediate/metadata"


conn = boto3.client('s3')
# contents = conn.list_objects(Bucket=bucket, Prefix=prefix)['Contents']

In [8]:
condition_categories = ['ano', 'ben',
     'bld',
 'brn',
 'cad',
 'can',
 'cer',
 'cir',
 'dia',
 'dig',
 'end',
 'gus',
 'hdz',
 'hiv',
 'inf',
 'inj',
 'men',
 'mus',
 'ner',
 'pre',
 'rar',
 'res',
 'rsk',
 'skn',
 'sns',
 'sor',
 'trm',
 'vco']

feature_columns_dtype = {}
feature_columns_dtype['raw_cols'] = {'submcc_ano_cns_pmpm_ct': 'float64',
 'submcc_ano_dig_pmpm_ct': 'float64',
 'submcc_ano_gu_pmpm_ct': 'float64',
 'submcc_ano_hrt_pmpm_ct': 'float64',
 'submcc_ano_mus_pmpm_ct': 'float64',
 'submcc_ano_othr_pmpm_ct': 'float64',
 'submcc_ben_lymp_pmpm_ct': 'float64',
 'submcc_ben_ner_pmpm_ct': 'float64',
 'submcc_ben_othr_pmpm_ct': 'float64',
 'submcc_ben_unk_pmpm_ct': 'float64',
 'submcc_bld_anem_pmpm_ct': 'float64',
 'submcc_bld_othr_pmpm_ct': 'float64',
 'submcc_brn_acc_pmpm_ct': 'float64',
 'submcc_brn_othr_pmpm_ct': 'float64',
 'submcc_cad_ang_pmpm_ct': 'float64',
 'submcc_cad_ashd_pmpm_ct': 'float64',
 'submcc_cad_cabg_pmpm_ct': 'float64',
 'submcc_cad_fh/ho_pmpm_ct': 'float64',
 'submcc_cad_isch_pmpm_ct': 'float64',
 'submcc_cad_mi_pmpm_ct': 'float64',
 'submcc_cad_ptca_pmpm_ct': 'float64',
 'submcc_can_brst_pmpm_ct': 'float64',
 'submcc_can_dig_pmpm_ct': 'float64',
 'submcc_can_end_pmpm_ct': 'float64',
 'submcc_can_gu_pmpm_ct': 'float64',
 'submcc_can_h/n_pmpm_ct': 'float64',
 'submcc_can_h/o_pmpm_ct': 'float64',
 'submcc_can_leuk_pmpm_ct': 'float64',
 'submcc_can_lymp_pmpm_ct': 'float64',
 'submcc_can_ner_pmpm_ct': 'float64',
 'submcc_can_othr_pmpm_ct': 'float64',
 'submcc_can_res_pmpm_ct': 'float64',
 'submcc_can_sec_pmpm_ct': 'float64',
 'submcc_can_skn_pmpm_ct': 'float64',
 'submcc_cer_hem_pmpm_ct': 'float64',
 'submcc_cer_occ_pmpm_ct': 'float64',
 'submcc_cer_seq_pmpm_ct': 'float64',
 'submcc_cer_tia_pmpm_ct': 'float64',
 'submcc_cir_anur_pmpm_ct': 'float64',
 'submcc_cir_art_pmpm_ct': 'float64',
 'submcc_cir_hbp_pmpm_ct': 'float64',
 'submcc_cir_othr_pmpm_ct': 'float64',
 'submcc_dia_eye_pmpm_ct': 'float64',
 'submcc_dia_nep_pmpm_ct': 'float64',
 'submcc_dia_neu_pmpm_ct': 'float64',
 'submcc_dia_othr_pmpm_ct': 'float64',
 'submcc_dia_pvd_pmpm_ct': 'float64',
 'submcc_dig_lgi_pmpm_ct': 'float64',
 'submcc_dig_liv_pmpm_ct': 'float64',
 'submcc_dig_othr_pmpm_ct': 'float64',
 'submcc_dig_p/b_pmpm_ct': 'float64',
 'submcc_dig_ugi_pmpm_ct': 'float64',
 'submcc_end_gld_pmpm_ct': 'float64',
 'submcc_end_meta_pmpm_ct': 'float64',
 'submcc_end_nutr_pmpm_ct': 'float64',
 'submcc_end_othr_pmpm_ct': 'float64',
 'submcc_end_thy_pmpm_ct': 'float64',
 'submcc_gus_brst_pmpm_ct': 'float64',
 'submcc_gus_fem_pmpm_ct': 'float64',
 'submcc_gus_kub_pmpm_ct': 'float64',
 'submcc_gus_male_pmpm_ct': 'float64',
 'submcc_gus_othr_pmpm_ct': 'float64',
 'submcc_hdz_arrh_pmpm_ct': 'float64',
 'submcc_hdz_it_i_pmpm_ct': 'float64',
 'submcc_hdz_it_is_pmpm_ct': 'float64',
 'submcc_hdz_myop_pmpm_ct': 'float64',
 'submcc_hdz_othr_pmpm_ct': 'float64',
 'submcc_hdz_surg_pmpm_ct': 'float64',
 'submcc_hdz_valv_pmpm_ct': 'float64',
 'submcc_hiv_kapo_pmpm_ct': 'float64',
 'submcc_hiv_othr_pmpm_ct': 'float64',
 'submcc_hiv_pcp_pmpm_ct': 'float64',
 'submcc_inf_cand_pmpm_ct': 'float64',
 'submcc_inf_men_pmpm_ct': 'float64',
 'submcc_inf_myco_pmpm_ct': 'float64',
 'submcc_inf_othr_pmpm_ct': 'float64',
 'submcc_inf_sep_pmpm_ct': 'float64',
 'submcc_inj_comp_pmpm_ct': 'float64',
 'submcc_inj_drug_pmpm_ct': 'float64',
 'submcc_inj_org_pmpm_ct': 'float64',
 'submcc_inj_othr_pmpm_ct': 'float64',
 'submcc_men_abus_pmpm_ct': 'float64',
 'submcc_men_alco_pmpm_ct': 'float64',
 'submcc_men_depr_pmpm_ct': 'float64',
 'submcc_men_othr_pmpm_ct': 'float64',
 'submcc_men_schz_pmpm_ct': 'float64',
 'submcc_mus_arth_pmpm_ct': 'float64',
 'submcc_mus_atrp_pmpm_ct': 'float64',
 'submcc_mus_back_pmpm_ct': 'float64',
 'submcc_mus_form_pmpm_ct': 'float64',
 'submcc_mus_inf_pmpm_ct': 'float64',
 'submcc_mus_jnt_pmpm_ct': 'float64',
 'submcc_mus_oste_pmpm_ct': 'float64',
 'submcc_mus_othr_pmpm_ct': 'float64',
 'submcc_mus_soft_pmpm_ct': 'float64',
 'submcc_neo_fh/ho_pmpm_ct': 'float64',
 'submcc_ner_deg_pmpm_ct': 'float64',
 'submcc_ner_epil_pmpm_ct': 'float64',
 'submcc_ner_infl_pmpm_ct': 'float64',
 'submcc_ner_migr_pmpm_ct': 'float64',
 'submcc_ner_othr_pmpm_ct': 'float64',
 'submcc_pre_care_pmpm_ct': 'float64',
 'submcc_pre_com_pmpm_ct': 'float64',
 'submcc_pre_del_pmpm_ct': 'float64',
 'submcc_pre_ect_pmpm_ct': 'float64',
 'submcc_pre_l/d_pmpm_ct': 'float64',
 'submcc_pre_mul_pmpm_ct': 'float64',
 'submcc_pre_othr_pmpm_ct': 'float64',
 'submcc_rar_als_pmpm_ct': 'float64',
 'submcc_rar_cf_pmpm_ct': 'float64',
 'submcc_rar_cid_pmpm_ct': 'float64',
 'submcc_rar_drm_pmpm_ct': 'float64',
 'submcc_rar_hem_pmpm_ct': 'float64',
 'submcc_rar_lup_pmpm_ct': 'float64',
 'submcc_rar_mg_pmpm_ct': 'float64',
 'submcc_rar_ms_pmpm_ct': 'float64',
 'submcc_rar_othr_pmpm_ct': 'float64',
 'submcc_rar_par_pmpm_ct': 'float64',
 'submcc_rar_pol_pmpm_ct': 'float64',
 'submcc_rar_ra_pmpm_ct': 'float64',
 'submcc_rar_sca_pmpm_ct': 'float64',
 'submcc_rar_scl_pmpm_ct': 'float64',
 'submcc_res_alg_pmpm_ct': 'float64',
 'submcc_res_asth_pmpm_ct': 'float64',
 'submcc_res_copd_pmpm_ct': 'float64',
 'submcc_res_fail_pmpm_ct': 'float64',
 'submcc_res_inf_pmpm_ct': 'float64',
 'submcc_res_othr_pmpm_ct': 'float64',
 'submcc_rsk_an_pmpm_ct': 'float64',
 'submcc_rsk_chol_pmpm_ct': 'float64',
 'submcc_rsk_coag_pmpm_ct': 'float64',
 'submcc_rsk_fh/h_pmpm_ct': 'float64',
 'submcc_rsk_fh/ho_pmpm_ct': 'float64',
 'submcc_rsk_glu_pmpm_ct': 'float64',
 'submcc_rsk_obe_pmpm_ct': 'float64',
 'submcc_rsk_othr_pmpm_ct': 'float64',
 'submcc_rsk_pcos_pmpm_ct': 'float64',
 'submcc_rsk_smok_pmpm_ct': 'float64',
 'submcc_rsk_synx_pmpm_ct': 'float64',
 'submcc_skn_inf_pmpm_ct': 'float64',
 'submcc_skn_othr_pmpm_ct': 'float64',
 'submcc_sns_abd_pmpm_ct': 'float64',
 'submcc_sns_chst_pmpm_ct': 'float64',
 'submcc_sns_coma_pmpm_ct': 'float64',
 'submcc_sns_cons_pmpm_ct': 'float64',
 'submcc_sns_dth_pmpm_ct': 'float64',
 'submcc_sns_othr_pmpm_ct': 'float64',
 'submcc_sor_ear_pmpm_ct': 'float64',
 'submcc_sor_eye_pmpm_ct': 'float64',
 'submcc_trm_brn_pmpm_ct': 'float64',
 'submcc_trm_f/n_pmpm_ct': 'float64',
 'submcc_trm_fxu_pmpm_ct': 'float64',
 'submcc_trm_fxul_pmpm_ct': 'float64',
 'submcc_trm_hip_pmpm_ct': 'float64',
 'submcc_trm_prly_pmpm_ct': 'float64',
 'submcc_trm_skul_pmpm_ct': 'float64',
 'submcc_trm_spfx_pmpm_ct': 'float64',
 'submcc_trm_spnj_pmpm_ct': 'float64',
 'submcc_vco_care_pmpm_ct': 'float64',
 'submcc_vco_end_pmpm_ct': 'float64',
 'submcc_vco_exam_pmpm_ct': 'float64',
 'submcc_vco_othr_pmpm_ct': 'float64',
 'submcc_vco_vac_pmpm_ct': 'float64',
 'fci_score': 'float64',
 'dcsi_score': 'float64',
 'cci_score': 'float64',
 'credit_bal_1stmtgcredit_60dpd': 'float64',
 'credit_bal_agencyfirstmtg_60dpd': 'float64',
 'credit_bal_heloc_60dpd': 'float64',
 'credit_bal_nonagnfirstmtg_60dpd': 'float64',
 'credit_bal_nonmtgcredit_60dpd': 'float64',
 'credit_bal_studentloan_60dpd': 'float64',
 'credit_bal_totalallcredit_60dpd': 'float64',
 'credit_bal_autobank': 'float64',
 'credit_bal_autofinance': 'float64',
 'credit_bal_consumerfinance': 'float64',
 'credit_minmob_mtgcredit': 'float64',
 'submcc_ano_cns_ind': 'category',
 'submcc_ano_dig_ind': 'category',
 'submcc_ano_gu_ind': 'category',
 'submcc_ano_hrt_ind': 'category',
 'submcc_ano_mus_ind': 'category',
 'submcc_ano_othr_ind': 'category',
 'submcc_ben_lymp_ind': 'category',
 'submcc_ben_ner_ind': 'category',
 'submcc_ben_othr_ind': 'category',
 'submcc_ben_unk_ind': 'category',
 'submcc_bld_anem_ind': 'category',
 'submcc_bld_othr_ind': 'category',
 'submcc_brn_acc_ind': 'category',
 'submcc_brn_othr_ind': 'category',
 'submcc_cad_ang_ind': 'category',
 'submcc_cad_ashd_ind': 'category',
 'submcc_cad_cabg_ind': 'category',
 'submcc_cad_fh/ho_ind': 'category',
 'submcc_cad_isch_ind': 'category',
 'submcc_cad_mi_ind': 'category',
 'submcc_cad_ptca_ind': 'category',
 'submcc_can_brst_ind': 'category',
 'submcc_can_dig_ind': 'category',
 'submcc_can_end_ind': 'category',
 'submcc_can_gu_ind': 'category',
 'submcc_can_h/n_ind': 'category',
 'submcc_can_h/o_ind': 'category',
 'submcc_can_leuk_ind': 'category',
 'submcc_can_lymp_ind': 'category',
 'submcc_can_ner_ind': 'category',
 'submcc_can_othr_ind': 'category',
 'submcc_can_res_ind': 'category',
 'submcc_can_sec_ind': 'category',
 'submcc_can_skn_ind': 'category',
 'submcc_cer_hem_ind': 'category',
 'submcc_cer_occ_ind': 'category',
 'submcc_cer_seq_ind': 'category',
 'submcc_cer_tia_ind': 'category',
 'submcc_cir_anur_ind': 'category',
 'submcc_cir_art_ind': 'category',
 'submcc_cir_hbp_ind': 'category',
 'submcc_cir_othr_ind': 'category',
 'submcc_dia_eye_ind': 'category',
 'submcc_dia_nep_ind': 'category',
 'submcc_dia_neu_ind': 'category',
 'submcc_dia_othr_ind': 'category',
 'submcc_dia_pvd_ind': 'category',
 'submcc_dig_lgi_ind': 'category',
 'submcc_dig_liv_ind': 'category',
 'submcc_dig_othr_ind': 'category',
 'submcc_dig_p/b_ind': 'category',
 'submcc_dig_ugi_ind': 'category',
 'submcc_end_gld_ind': 'category',
 'submcc_end_meta_ind': 'category',
 'submcc_end_nutr_ind': 'category',
 'submcc_end_othr_ind': 'category',
 'submcc_end_thy_ind': 'category',
 'submcc_gus_brst_ind': 'category',
 'submcc_gus_fem_ind': 'category',
 'submcc_gus_kub_ind': 'category',
 'submcc_gus_male_ind': 'category',
 'submcc_gus_othr_ind': 'category',
 'submcc_hdz_arrh_ind': 'category',
 'submcc_hdz_it_i_ind': 'category',
 'submcc_hdz_it_is_ind': 'category',
 'submcc_hdz_myop_ind': 'category',
 'submcc_hdz_othr_ind': 'category',
 'submcc_hdz_surg_ind': 'category',
 'submcc_hdz_valv_ind': 'category',
 'submcc_hiv_kapo_ind': 'category',
 'submcc_hiv_othr_ind': 'category',
 'submcc_hiv_pcp_ind': 'category',
 'submcc_inf_cand_ind': 'category',
 'submcc_inf_men_ind': 'category',
 'submcc_inf_myco_ind': 'category',
 'submcc_inf_othr_ind': 'category',
 'submcc_inf_sep_ind': 'category',
 'submcc_inj_comp_ind': 'category',
 'submcc_inj_drug_ind': 'category',
 'submcc_inj_org_ind': 'category',
 'submcc_inj_othr_ind': 'category',
 'submcc_men_abus_ind': 'category',
 'submcc_men_alco_ind': 'category',
 'submcc_men_depr_ind': 'category',
 'submcc_men_othr_ind': 'category',
 'submcc_men_schz_ind': 'category',
 'submcc_mus_arth_ind': 'category',
 'submcc_mus_atrp_ind': 'category',
 'submcc_mus_back_ind': 'category',
 'submcc_mus_form_ind': 'category',
 'submcc_mus_inf_ind': 'category',
 'submcc_mus_jnt_ind': 'category',
 'submcc_mus_oste_ind': 'category',
 'submcc_mus_othr_ind': 'category',
 'submcc_mus_soft_ind': 'category',
 'submcc_neo_fh/ho_ind': 'category',
 'submcc_ner_deg_ind': 'category',
 'submcc_ner_epil_ind': 'category',
 'submcc_ner_infl_ind': 'category',
 'submcc_ner_migr_ind': 'category',
 'submcc_ner_othr_ind': 'category',
 'submcc_pre_care_ind': 'category',
 'submcc_pre_com_ind': 'category',
 'submcc_pre_del_ind': 'category',
 'submcc_pre_ect_ind': 'category',
 'submcc_pre_l/d_ind': 'category',
 'submcc_pre_mul_ind': 'category',
 'submcc_pre_othr_ind': 'category',
 'submcc_rar_als_ind': 'category',
 'submcc_rar_cf_ind': 'category',
 'submcc_rar_cid_ind': 'category',
 'submcc_rar_drm_ind': 'category',
 'submcc_rar_hem_ind': 'category',
 'submcc_rar_lup_ind': 'category',
 'submcc_rar_mg_ind': 'category',
 'submcc_rar_ms_ind': 'category',
 'submcc_rar_othr_ind': 'category',
 'submcc_rar_par_ind': 'category',
 'submcc_rar_pol_ind': 'category',
 'submcc_rar_ra_ind': 'category',
 'submcc_rar_sca_ind': 'category',
 'submcc_rar_scl_ind': 'category',
 'submcc_res_alg_ind': 'category',
 'submcc_res_asth_ind': 'category',
 'submcc_res_copd_ind': 'category',
 'submcc_res_fail_ind': 'category',
 'submcc_res_inf_ind': 'category',
 'submcc_res_othr_ind': 'category',
 'submcc_rsk_an_ind': 'category',
 'submcc_rsk_chol_ind': 'category',
 'submcc_rsk_coag_ind': 'category',
 'submcc_rsk_fh/h_ind': 'category',
 'submcc_rsk_fh/ho_ind': 'category',
 'submcc_rsk_glu_ind': 'category',
 'submcc_rsk_obe_ind': 'category',
 'submcc_rsk_othr_ind': 'category',
 'submcc_rsk_pcos_ind': 'category',
 'submcc_rsk_smok_ind': 'category',
 'submcc_rsk_synx_ind': 'category',
 'submcc_skn_inf_ind': 'category',
 'submcc_skn_othr_ind': 'category',
 'submcc_sns_abd_ind': 'category',
 'submcc_sns_chst_ind': 'category',
 'submcc_sns_coma_ind': 'category',
 'submcc_sns_cons_ind': 'category',
 'submcc_sns_dth_ind': 'category',
 'submcc_sns_othr_ind': 'category',
 'submcc_sor_ear_ind': 'category',
 'submcc_sor_eye_ind': 'category',
 'submcc_trm_brn_ind': 'category',
 'submcc_trm_f/n_ind': 'category',
 'submcc_trm_fxu_ind': 'category',
 'submcc_trm_fxul_ind': 'category',
 'submcc_trm_hip_ind': 'category',
 'submcc_trm_prly_ind': 'category',
 'submcc_trm_skul_ind': 'category',
 'submcc_trm_spfx_ind': 'category',
 'submcc_trm_spnj_ind': 'category',
 'submcc_vco_care_ind': 'category',
 'submcc_vco_end_ind': 'category',
 'submcc_vco_exam_ind': 'category',
 'submcc_vco_othr_ind': 'category',
 'submcc_vco_vac_ind': 'category',
 'cmsd2_can_unc_neo/plycyth/myelo_ind': 'category',
 'cmsd2_eye_blindness_ind': 'category',
 'cmsd2_gus_m_genital_ind': 'category',
 'cmsd2_men_mad_ind': 'category',
 'cmsd2_men_men_substance_ind': 'category',
 'cmsd2_mus_polyarthropath_ind': 'category',
 'cmsd2_mus_spondylopath_ind': 'category',
 'cmsd2_skn_radiation_ind': 'category',
 'cmsd2_sns_general_ind': 'category'}

feature_columns_dtype['curated_cols'] = {'credit_num_new_accounts': 'float64',
 'credit_num_collections': 'float64',
 'ano_ind_sum': 'float64',
 'ano_pmpm_sum': 'float64',
 'ben_ind_sum': 'float64',
 'ben_pmpm_sum': 'float64',
 'bld_ind_sum': 'float64',
 'bld_pmpm_sum': 'float64',
 'brn_ind_sum': 'float64',
 'brn_pmpm_sum': 'float64',
 'cad_ind_sum': 'float64',
 'cad_pmpm_sum': 'float64',
 'can_ind_sum': 'float64',
 'can_pmpm_sum': 'float64',
 'cer_ind_sum': 'float64',
 'cer_pmpm_sum': 'float64',
 'cir_ind_sum': 'float64',
 'cir_pmpm_sum': 'float64',
 'dia_ind_sum': 'float64',
 'dia_pmpm_sum': 'float64',
 'dig_ind_sum': 'float64',
 'dig_pmpm_sum': 'float64',
 'end_ind_sum': 'float64',
 'end_pmpm_sum': 'float64',
 'gus_ind_sum': 'float64',
 'gus_pmpm_sum': 'float64',
 'hdz_ind_sum': 'float64',
 'hdz_pmpm_sum': 'float64',
 'hiv_ind_sum': 'float64',
 'hiv_pmpm_sum': 'float64',
 'inf_ind_sum': 'float64',
 'inf_pmpm_sum': 'float64',
 'inj_ind_sum': 'float64',
 'inj_pmpm_sum': 'float64',
 'men_ind_sum': 'float64',
 'men_pmpm_sum': 'float64',
 'mus_ind_sum': 'float64',
 'mus_pmpm_sum': 'float64',
 'ner_ind_sum': 'float64',
 'ner_pmpm_sum': 'float64',
 'pre_ind_sum': 'float64',
 'pre_pmpm_sum': 'float64',
 'rar_ind_sum': 'float64',
 'rar_pmpm_sum': 'float64',
 'res_ind_sum': 'float64',
 'res_pmpm_sum': 'float64',
 'rsk_ind_sum': 'float64',
 'rsk_pmpm_sum': 'float64',
 'skn_ind_sum': 'float64',
 'skn_pmpm_sum': 'float64',
 'sns_ind_sum': 'float64',
 'sns_pmpm_sum': 'float64',
 'sor_ind_sum': 'float64',
 'sor_pmpm_sum': 'float64',
 'trm_ind_sum': 'float64',
 'trm_pmpm_sum': 'float64',
 'vco_ind_sum': 'float64',
 'vco_pmpm_sum': 'float64',
 'total_ind': 'float64',
 'total_pmpm': 'float64',
 'service_bool': 'category'
} 

feature_column_cols = {}
feature_column_cols['condition'] = {}
feature_column_cols['condition']['categorical_cols'] = ['submcc_ano_cns_ind',
 'submcc_ano_dig_ind',
 'submcc_ano_gu_ind',
 'submcc_ano_hrt_ind',
 'submcc_ano_mus_ind',
 'submcc_ano_othr_ind',
 'submcc_ben_lymp_ind',
 'submcc_ben_ner_ind',
 'submcc_ben_othr_ind',
 'submcc_ben_unk_ind',
 'submcc_bld_anem_ind',
 'submcc_bld_othr_ind',
 'submcc_brn_acc_ind',
 'submcc_brn_othr_ind',
 'submcc_cad_ang_ind',
 'submcc_cad_ashd_ind',
 'submcc_cad_cabg_ind',
 'submcc_cad_fh/ho_ind',
 'submcc_cad_isch_ind',
 'submcc_cad_mi_ind',
 'submcc_cad_ptca_ind',
 'submcc_can_brst_ind',
 'submcc_can_dig_ind',
 'submcc_can_end_ind',
 'submcc_can_gu_ind',
 'submcc_can_h/n_ind',
 'submcc_can_h/o_ind',
 'submcc_can_leuk_ind',
 'submcc_can_lymp_ind',
 'submcc_can_ner_ind',
 'submcc_can_othr_ind',
 'submcc_can_res_ind',
 'submcc_can_sec_ind',
 'submcc_can_skn_ind',
 'submcc_cer_hem_ind',
 'submcc_cer_occ_ind',
 'submcc_cer_seq_ind',
 'submcc_cer_tia_ind',
 'submcc_cir_anur_ind',
 'submcc_cir_art_ind',
 'submcc_cir_hbp_ind',
 'submcc_cir_othr_ind',
 'submcc_dia_eye_ind',
 'submcc_dia_nep_ind',
 'submcc_dia_neu_ind',
 'submcc_dia_othr_ind',
 'submcc_dia_pvd_ind',
 'submcc_dig_lgi_ind',
 'submcc_dig_liv_ind',
 'submcc_dig_othr_ind',
 'submcc_dig_p/b_ind',
 'submcc_dig_ugi_ind',
 'submcc_end_gld_ind',
 'submcc_end_meta_ind',
 'submcc_end_nutr_ind',
 'submcc_end_othr_ind',
 'submcc_end_thy_ind',
 'submcc_gus_brst_ind',
 'submcc_gus_fem_ind',
 'submcc_gus_kub_ind',
 'submcc_gus_male_ind',
 'submcc_gus_othr_ind',
 'submcc_hdz_arrh_ind',
 'submcc_hdz_it_i_ind',
 'submcc_hdz_it_is_ind',
 'submcc_hdz_myop_ind',
 'submcc_hdz_othr_ind',
 'submcc_hdz_surg_ind',
 'submcc_hdz_valv_ind',
 'submcc_hiv_kapo_ind',
 'submcc_hiv_othr_ind',
 'submcc_hiv_pcp_ind',
 'submcc_inf_cand_ind',
 'submcc_inf_men_ind',
 'submcc_inf_myco_ind',
 'submcc_inf_othr_ind',
 'submcc_inf_sep_ind',
 'submcc_inj_comp_ind',
 'submcc_inj_drug_ind',
 'submcc_inj_org_ind',
 'submcc_inj_othr_ind',
 'submcc_men_abus_ind',
 'submcc_men_alco_ind',
 'submcc_men_depr_ind',
 'submcc_men_othr_ind',
 'submcc_men_schz_ind',
 'submcc_mus_arth_ind',
 'submcc_mus_atrp_ind',
 'submcc_mus_back_ind',
 'submcc_mus_form_ind',
 'submcc_mus_inf_ind',
 'submcc_mus_jnt_ind',
 'submcc_mus_oste_ind',
 'submcc_mus_othr_ind',
 'submcc_mus_soft_ind',
 'submcc_neo_fh/ho_ind',
 'submcc_ner_deg_ind',
 'submcc_ner_epil_ind',
 'submcc_ner_infl_ind',
 'submcc_ner_migr_ind',
 'submcc_ner_othr_ind',
 'submcc_pre_care_ind',
 'submcc_pre_com_ind',
 'submcc_pre_del_ind',
 'submcc_pre_ect_ind',
 'submcc_pre_l/d_ind',
 'submcc_pre_mul_ind',
 'submcc_pre_othr_ind',
 'submcc_rar_als_ind',
 'submcc_rar_cf_ind',
 'submcc_rar_cid_ind',
 'submcc_rar_drm_ind',
 'submcc_rar_hem_ind',
 'submcc_rar_lup_ind',
 'submcc_rar_mg_ind',
 'submcc_rar_ms_ind',
 'submcc_rar_othr_ind',
 'submcc_rar_par_ind',
 'submcc_rar_pol_ind',
 'submcc_rar_ra_ind',
 'submcc_rar_sca_ind',
 'submcc_rar_scl_ind',
 'submcc_res_alg_ind',
 'submcc_res_asth_ind',
 'submcc_res_copd_ind',
 'submcc_res_fail_ind',
 'submcc_res_inf_ind',
 'submcc_res_othr_ind',
 'submcc_rsk_an_ind',
 'submcc_rsk_chol_ind',
 'submcc_rsk_coag_ind',
 'submcc_rsk_fh/h_ind',
 'submcc_rsk_fh/ho_ind',
 'submcc_rsk_glu_ind',
 'submcc_rsk_obe_ind',
 'submcc_rsk_othr_ind',
 'submcc_rsk_pcos_ind',
 'submcc_rsk_smok_ind',
 'submcc_rsk_synx_ind',
 'submcc_skn_inf_ind',
 'submcc_skn_othr_ind',
 'submcc_sns_abd_ind',
 'submcc_sns_chst_ind',
 'submcc_sns_coma_ind',
 'submcc_sns_cons_ind',
 'submcc_sns_dth_ind',
 'submcc_sns_othr_ind',
 'submcc_sor_ear_ind',
 'submcc_sor_eye_ind',
 'submcc_trm_brn_ind',
 'submcc_trm_f/n_ind',
 'submcc_trm_fxu_ind',
 'submcc_trm_fxul_ind',
 'submcc_trm_hip_ind',
 'submcc_trm_prly_ind',
 'submcc_trm_skul_ind',
 'submcc_trm_spfx_ind',
 'submcc_trm_spnj_ind',
 'submcc_vco_care_ind',
 'submcc_vco_end_ind',
 'submcc_vco_exam_ind',
 'submcc_vco_othr_ind',
 'submcc_vco_vac_ind',
 'cmsd2_can_unc_neo/plycyth/myelo_ind',
 'cmsd2_eye_blindness_ind',
 'cmsd2_gus_m_genital_ind',
 'cmsd2_men_mad_ind',
 'cmsd2_men_men_substance_ind',
 'cmsd2_mus_polyarthropath_ind',
 'cmsd2_mus_spondylopath_ind',
 'cmsd2_skn_radiation_ind',
 'cmsd2_sns_general_ind']

feature_curated_cols = {}
feature_curated_cols['condition'] = {}
feature_curated_cols['condition']['categorical_cols'] = [ 'service_bool']


feature_column_cols['condition']['numerical_cols'] = ['submcc_ano_cns_pmpm_ct',
 'submcc_ano_dig_pmpm_ct',
 'submcc_ano_gu_pmpm_ct',
 'submcc_ano_hrt_pmpm_ct',
 'submcc_ano_mus_pmpm_ct',
 'submcc_ano_othr_pmpm_ct',
 'submcc_ben_lymp_pmpm_ct',
 'submcc_ben_ner_pmpm_ct',
 'submcc_ben_othr_pmpm_ct',
 'submcc_ben_unk_pmpm_ct',
 'submcc_bld_anem_pmpm_ct',
 'submcc_bld_othr_pmpm_ct',
 'submcc_brn_acc_pmpm_ct',
 'submcc_brn_othr_pmpm_ct',
 'submcc_cad_ang_pmpm_ct',
 'submcc_cad_ashd_pmpm_ct',
 'submcc_cad_cabg_pmpm_ct',
 'submcc_cad_fh/ho_pmpm_ct',
 'submcc_cad_isch_pmpm_ct',
 'submcc_cad_mi_pmpm_ct',
 'submcc_cad_ptca_pmpm_ct',
 'submcc_can_brst_pmpm_ct',
 'submcc_can_dig_pmpm_ct',
 'submcc_can_end_pmpm_ct',
 'submcc_can_gu_pmpm_ct',
 'submcc_can_h/n_pmpm_ct',
 'submcc_can_h/o_pmpm_ct',
 'submcc_can_leuk_pmpm_ct',
 'submcc_can_lymp_pmpm_ct',
 'submcc_can_ner_pmpm_ct',
 'submcc_can_othr_pmpm_ct',
 'submcc_can_res_pmpm_ct',
 'submcc_can_sec_pmpm_ct',
 'submcc_can_skn_pmpm_ct',
 'submcc_cer_hem_pmpm_ct',
 'submcc_cer_occ_pmpm_ct',
 'submcc_cer_seq_pmpm_ct',
 'submcc_cer_tia_pmpm_ct',
 'submcc_cir_anur_pmpm_ct',
 'submcc_cir_art_pmpm_ct',
 'submcc_cir_hbp_pmpm_ct',
 'submcc_cir_othr_pmpm_ct',
 'submcc_dia_eye_pmpm_ct',
 'submcc_dia_nep_pmpm_ct',
 'submcc_dia_neu_pmpm_ct',
 'submcc_dia_othr_pmpm_ct',
 'submcc_dia_pvd_pmpm_ct',
 'submcc_dig_lgi_pmpm_ct',
 'submcc_dig_liv_pmpm_ct',
 'submcc_dig_othr_pmpm_ct',
 'submcc_dig_p/b_pmpm_ct',
 'submcc_dig_ugi_pmpm_ct',
 'submcc_end_gld_pmpm_ct',
 'submcc_end_meta_pmpm_ct',
 'submcc_end_nutr_pmpm_ct',
 'submcc_end_othr_pmpm_ct',
 'submcc_end_thy_pmpm_ct',
 'submcc_gus_brst_pmpm_ct',
 'submcc_gus_fem_pmpm_ct',
 'submcc_gus_kub_pmpm_ct',
 'submcc_gus_male_pmpm_ct',
 'submcc_gus_othr_pmpm_ct',
 'submcc_hdz_arrh_pmpm_ct',
 'submcc_hdz_it_i_pmpm_ct',
 'submcc_hdz_it_is_pmpm_ct',
 'submcc_hdz_myop_pmpm_ct',
 'submcc_hdz_othr_pmpm_ct',
 'submcc_hdz_surg_pmpm_ct',
 'submcc_hdz_valv_pmpm_ct',
 'submcc_hiv_kapo_pmpm_ct',
 'submcc_hiv_othr_pmpm_ct',
 'submcc_hiv_pcp_pmpm_ct',
 'submcc_inf_cand_pmpm_ct',
 'submcc_inf_men_pmpm_ct',
 'submcc_inf_myco_pmpm_ct',
 'submcc_inf_othr_pmpm_ct',
 'submcc_inf_sep_pmpm_ct',
 'submcc_inj_comp_pmpm_ct',
 'submcc_inj_drug_pmpm_ct',
 'submcc_inj_org_pmpm_ct',
 'submcc_inj_othr_pmpm_ct',
 'submcc_men_abus_pmpm_ct',
 'submcc_men_alco_pmpm_ct',
 'submcc_men_depr_pmpm_ct',
 'submcc_men_othr_pmpm_ct',
 'submcc_men_schz_pmpm_ct',
 'submcc_mus_arth_pmpm_ct',
 'submcc_mus_atrp_pmpm_ct',
 'submcc_mus_back_pmpm_ct',
 'submcc_mus_form_pmpm_ct',
 'submcc_mus_inf_pmpm_ct',
 'submcc_mus_jnt_pmpm_ct',
 'submcc_mus_oste_pmpm_ct',
 'submcc_mus_othr_pmpm_ct',
 'submcc_mus_soft_pmpm_ct',
 'submcc_neo_fh/ho_pmpm_ct',
 'submcc_ner_deg_pmpm_ct',
 'submcc_ner_epil_pmpm_ct',
 'submcc_ner_infl_pmpm_ct',
 'submcc_ner_migr_pmpm_ct',
 'submcc_ner_othr_pmpm_ct',
 'submcc_pre_care_pmpm_ct',
 'submcc_pre_com_pmpm_ct',
 'submcc_pre_del_pmpm_ct',
 'submcc_pre_ect_pmpm_ct',
 'submcc_pre_l/d_pmpm_ct',
 'submcc_pre_mul_pmpm_ct',
 'submcc_pre_othr_pmpm_ct',
 'submcc_rar_als_pmpm_ct',
 'submcc_rar_cf_pmpm_ct',
 'submcc_rar_cid_pmpm_ct',
 'submcc_rar_drm_pmpm_ct',
 'submcc_rar_hem_pmpm_ct',
 'submcc_rar_lup_pmpm_ct',
 'submcc_rar_mg_pmpm_ct',
 'submcc_rar_ms_pmpm_ct',
 'submcc_rar_othr_pmpm_ct',
 'submcc_rar_par_pmpm_ct',
 'submcc_rar_pol_pmpm_ct',
 'submcc_rar_ra_pmpm_ct',
 'submcc_rar_sca_pmpm_ct',
 'submcc_rar_scl_pmpm_ct',
 'submcc_res_alg_pmpm_ct',
 'submcc_res_asth_pmpm_ct',
 'submcc_res_copd_pmpm_ct',
 'submcc_res_fail_pmpm_ct',
 'submcc_res_inf_pmpm_ct',
 'submcc_res_othr_pmpm_ct',
 'submcc_rsk_an_pmpm_ct',
 'submcc_rsk_chol_pmpm_ct',
 'submcc_rsk_coag_pmpm_ct',
 'submcc_rsk_fh/h_pmpm_ct',
 'submcc_rsk_fh/ho_pmpm_ct',
 'submcc_rsk_glu_pmpm_ct',
 'submcc_rsk_obe_pmpm_ct',
 'submcc_rsk_othr_pmpm_ct',
 'submcc_rsk_pcos_pmpm_ct',
 'submcc_rsk_smok_pmpm_ct',
 'submcc_rsk_synx_pmpm_ct',
 'submcc_skn_inf_pmpm_ct',
 'submcc_skn_othr_pmpm_ct',
 'submcc_sns_abd_pmpm_ct',
 'submcc_sns_chst_pmpm_ct',
 'submcc_sns_coma_pmpm_ct',
 'submcc_sns_cons_pmpm_ct',
 'submcc_sns_dth_pmpm_ct',
 'submcc_sns_othr_pmpm_ct',
 'submcc_sor_ear_pmpm_ct',
 'submcc_sor_eye_pmpm_ct',
 'submcc_trm_brn_pmpm_ct',
 'submcc_trm_f/n_pmpm_ct',
 'submcc_trm_fxu_pmpm_ct',
 'submcc_trm_fxul_pmpm_ct',
 'submcc_trm_hip_pmpm_ct',
 'submcc_trm_prly_pmpm_ct',
 'submcc_trm_skul_pmpm_ct',
 'submcc_trm_spfx_pmpm_ct',
 'submcc_trm_spnj_pmpm_ct',
 'submcc_vco_care_pmpm_ct',
 'submcc_vco_end_pmpm_ct',
 'submcc_vco_exam_pmpm_ct',
 'submcc_vco_othr_pmpm_ct',
 'submcc_vco_vac_pmpm_ct',
 'fci_score',
 'dcsi_score',
 'cci_score',]

feature_curated_cols['condition']['numerical_cols'] = ['ano_ind_sum',
 'ano_pmpm_sum',
 'ben_ind_sum',
 'ben_pmpm_sum',
 'bld_ind_sum',
 'bld_pmpm_sum',
 'brn_ind_sum',
 'brn_pmpm_sum',
 'cad_ind_sum',
 'cad_pmpm_sum',
 'can_ind_sum',
 'can_pmpm_sum',
 'cer_ind_sum',
 'cer_pmpm_sum',
 'cir_ind_sum',
 'cir_pmpm_sum',
 'dia_ind_sum',
 'dia_pmpm_sum',
 'dig_ind_sum',
 'dig_pmpm_sum',
 'end_ind_sum',
 'end_pmpm_sum',
 'gus_ind_sum',
 'gus_pmpm_sum',
 'hdz_ind_sum',
 'hdz_pmpm_sum',
 'hiv_ind_sum',
 'hiv_pmpm_sum',
 'inf_ind_sum',
 'inf_pmpm_sum',
 'inj_ind_sum',
 'inj_pmpm_sum',
 'men_ind_sum',
 'men_pmpm_sum',
 'mus_ind_sum',
 'mus_pmpm_sum',
 'ner_ind_sum',
 'ner_pmpm_sum',
 'pre_ind_sum',
 'pre_pmpm_sum',
 'rar_ind_sum',
 'rar_pmpm_sum',
 'res_ind_sum',
 'res_pmpm_sum',
 'rsk_ind_sum',
 'rsk_pmpm_sum',
 'skn_ind_sum',
 'skn_pmpm_sum',
 'sns_ind_sum',
 'sns_pmpm_sum',
 'sor_ind_sum',
 'sor_pmpm_sum',
 'trm_ind_sum',
 'trm_pmpm_sum',
 'vco_ind_sum',
 'vco_pmpm_sum',
 'total_ind',
 'total_pmpm']

In [9]:
feature_column_cols['credit'] = {}
feature_column_cols['credit']['numerical_cols'] = ['credit_bal_1stmtgcredit_60dpd', 'credit_bal_agencyfirstmtg_60dpd', 
             'credit_bal_heloc_60dpd', 'credit_bal_nonagnfirstmtg_60dpd', 'credit_bal_nonmtgcredit_60dpd',
             'credit_bal_studentloan_60dpd', 'credit_bal_totalallcredit_60dpd', 'credit_bal_autobank',
             'credit_bal_autofinance', 'credit_bal_consumerfinance', 'credit_minmob_mtgcredit']

feature_curated_cols['credit'] = {}
feature_curated_cols['credit']['numerical_cols'] = ['credit_num_new_accounts', 'credit_num_collections']



In [5]:
label_column = "transportation_issues"
label_columns_dtype ={"tranportation_issues": "float64"}

In [6]:
credit_na_fill = {'credit_bal_1stmtgcredit_60dpd': 1269.43,
 'credit_bal_agencyfirstmtg_60dpd': 419.66,
 'credit_bal_heloc_60dpd': 71.17,
 'credit_bal_nonagnfirstmtg_60dpd': 811.61,
 'credit_bal_nonmtgcredit_60dpd': 2609.02,
 'credit_bal_studentloan_60dpd': 1254.13,
 'credit_bal_totalallcredit_60dpd': 3972.36,
 'credit_bal_autobank': 4914.82,
 'credit_bal_autofinance': 5279.56,
 'credit_bal_consumerfinance': 878.67,
 'credit_minmob_mtgcredit': 24.82,
 'credit_num_new_accounts': 0.44,
 'credit_num_collections': 0.05}

In [7]:
def sum_feature_generation(df, categories):
    for category in categories:
        new_ind_name = category + "_ind_sum"
        subset_cols_criteria = "_" + category + "_"
        subset_cols = [x for x in df.columns if subset_cols_criteria in x and '_ind' in x]
        df[new_ind_name] = df[subset_cols].apply(pd.to_numeric, errors='coerce').sum(axis=1)
        df[subset_cols] = df[subset_cols].astype("category")
        

        new_pmpm_name = category + "_pmpm_sum"
        subset_cols_criteria = "_" + category + "_"
        subset_cols = [x for x in df.columns if subset_cols_criteria in x and '_pmpm' in x]

        df[new_pmpm_name] = df[subset_cols].apply(pd.to_numeric, errors='coerce').sum(axis=1)
    return df

In [8]:
def total_sum_features(df):
    df['total_ind'] = df[df.columns[df.columns.str.contains('_ind_sum')]].sum(axis=1)
    df['total_pmpm'] = df[df.columns[df.columns.str.contains('_pmpm_sum')]].sum(axis=1)
    df['service_bool'] = np.where(df['total_ind'] == 0, 0, 1)
    return df

In [9]:
def credit_attributes(df):
    # data Transformation
    df['credit_num_new_accounts'] = df[['credit_num_autobank_new', 
                                                      'credit_num_autofinance_new',
                                                      'credit_num_consumerfinance_new',
                                                      'credit_num_mtgcredit_new']].astype(float).sum(axis=1)
    
    df['credit_num_collections'] = (df[['credit_num_mtg_collections',
                                                      'credit_num_totalallcredit_collections']].astype(float).sum(axis=1))/2
    
    
    
    credit_df = pd.concat([df[feature_column_cols['credit']['numerical_cols']],
                           df[feature_curated_cols['credit']['numerical_cols']]], axis=1)
    
    return credit_df

In [10]:
def credit_fillna(credit_df):
    for col in credit_df.columns:
        na_value = credit_na_fill[col]
        credit_df[col] = credit_df[col].fillna(na_value)
    return credit_df

In [11]:
contents = conn.list_objects(Bucket=bucket, Prefix=prefix_rawdata_train)['Contents']
rawfile_names = [key['Key'] for key in contents]

input_files = ["s3://" + bucket + '/' + x for x in rawfile_names if '.csv' in x]

input_files

['s3://humana-data/rawdata/original_raw_files/train/condition.csv',
 's3://humana-data/rawdata/original_raw_files/train/credit.csv',
 's3://humana-data/rawdata/original_raw_files/train/dependent.csv']

In [12]:
raw_data = [pd.read_csv(file) for file in input_files]

In [13]:
concat_data = pd.concat(raw_data, axis=1)

for col, type_ in feature_columns_dtype['raw_cols'].items():
    concat_data[col] = concat_data[col].astype(type_)

In [14]:

concat_data.columns = concat_data.columns.str.lower()

condition_df = concat_data[feature_column_cols['condition']['numerical_cols'] + feature_column_cols['condition']['categorical_cols']]
condition_df[feature_column_cols['condition']['categorical_cols']] = np.where(condition_df[feature_column_cols['condition']['categorical_cols']] != 0, 1,0)
condition_df = sum_feature_generation(df = condition_df, categories = condition_categories)
condition_fe_df = total_sum_features(df = condition_df)


In [15]:
credit_df = credit_attributes(df=concat_data)
credit_fe_df = credit_fillna(credit_df=credit_df)

In [16]:
df_fe = credit_fe_df.merge(condition_fe_df, how='left', left_index=True, right_index=True)

In [14]:
feature_columns_dtype

{'raw_cols': {'submcc_ano_cns_pmpm_ct': 'float64',
  'submcc_ano_dig_pmpm_ct': 'float64',
  'submcc_ano_gu_pmpm_ct': 'float64',
  'submcc_ano_hrt_pmpm_ct': 'float64',
  'submcc_ano_mus_pmpm_ct': 'float64',
  'submcc_ano_othr_pmpm_ct': 'float64',
  'submcc_ben_lymp_pmpm_ct': 'float64',
  'submcc_ben_ner_pmpm_ct': 'float64',
  'submcc_ben_othr_pmpm_ct': 'float64',
  'submcc_ben_unk_pmpm_ct': 'float64',
  'submcc_bld_anem_pmpm_ct': 'float64',
  'submcc_bld_othr_pmpm_ct': 'float64',
  'submcc_brn_acc_pmpm_ct': 'float64',
  'submcc_brn_othr_pmpm_ct': 'float64',
  'submcc_cad_ang_pmpm_ct': 'float64',
  'submcc_cad_ashd_pmpm_ct': 'float64',
  'submcc_cad_cabg_pmpm_ct': 'float64',
  'submcc_cad_fh/ho_pmpm_ct': 'float64',
  'submcc_cad_isch_pmpm_ct': 'float64',
  'submcc_cad_mi_pmpm_ct': 'float64',
  'submcc_cad_ptca_pmpm_ct': 'float64',
  'submcc_can_brst_pmpm_ct': 'float64',
  'submcc_can_dig_pmpm_ct': 'float64',
  'submcc_can_end_pmpm_ct': 'float64',
  'submcc_can_gu_pmpm_ct': 'float64',
  '

In [19]:
feature_dtypes_all = {}
for key in feature_columns_dtype.keys():
    feature_dtypes_all =  {**feature_dtypes_all, **feature_columns_dtype[key]}

In [20]:
feature_dtypes_all

{'submcc_ano_cns_pmpm_ct': 'float64',
 'submcc_ano_dig_pmpm_ct': 'float64',
 'submcc_ano_gu_pmpm_ct': 'float64',
 'submcc_ano_hrt_pmpm_ct': 'float64',
 'submcc_ano_mus_pmpm_ct': 'float64',
 'submcc_ano_othr_pmpm_ct': 'float64',
 'submcc_ben_lymp_pmpm_ct': 'float64',
 'submcc_ben_ner_pmpm_ct': 'float64',
 'submcc_ben_othr_pmpm_ct': 'float64',
 'submcc_ben_unk_pmpm_ct': 'float64',
 'submcc_bld_anem_pmpm_ct': 'float64',
 'submcc_bld_othr_pmpm_ct': 'float64',
 'submcc_brn_acc_pmpm_ct': 'float64',
 'submcc_brn_othr_pmpm_ct': 'float64',
 'submcc_cad_ang_pmpm_ct': 'float64',
 'submcc_cad_ashd_pmpm_ct': 'float64',
 'submcc_cad_cabg_pmpm_ct': 'float64',
 'submcc_cad_fh/ho_pmpm_ct': 'float64',
 'submcc_cad_isch_pmpm_ct': 'float64',
 'submcc_cad_mi_pmpm_ct': 'float64',
 'submcc_cad_ptca_pmpm_ct': 'float64',
 'submcc_can_brst_pmpm_ct': 'float64',
 'submcc_can_dig_pmpm_ct': 'float64',
 'submcc_can_end_pmpm_ct': 'float64',
 'submcc_can_gu_pmpm_ct': 'float64',
 'submcc_can_h/n_pmpm_ct': 'float64',
 '

In [18]:
for col, type_ in feature_dtypes_all.items():
    df_fe[col] = df_fe[col].astype(type_)

In [19]:
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Binarizer, StandardScaler, OneHotEncoder

# from sagemaker_containers.beta.framework import (
#     content_types, encoders, env, modules, transformer, worker)

In [20]:
numeric_transformer = make_pipeline(
    SimpleImputer(strategy='median'),
    StandardScaler())

In [21]:
categorical_transformer = make_pipeline(
        SimpleImputer(strategy='constant'),
        OneHotEncoder(handle_unknown='ignore'))

In [22]:
preprocessor = ColumnTransformer(transformers=[
            ("num", numeric_transformer, make_column_selector(dtype_exclude="category")),
            ("cat", categorical_transformer, make_column_selector(dtype_include="category"))])

preprocessor.fit(df_fe)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f1a41667cd0>),
                                ('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7f1a41667760>)])

In [ ]:
 joblib.dump(preprocessor, os.path.join(args.model_dir, "model.joblib"))

In [9]:
file_condition = "condition.parquet.gzip"

buffer = io.BytesIO()
s3 = boto3.resource('s3')
object_ = s3.Object('humana-data', '{}/{}'.format(prefix_rawdata_train, file_condition))
object_.download_fileobj(buffer)
condition_df = pd.read_parquet(buffer)


file_credit = "credit.parquet.gzip"

buffer = io.BytesIO()
s3 = boto3.resource('s3')
object_ = s3.Object('humana-data', '{}/{}'.format(prefix_rawdata_train, file_credit))
object_.download_fileobj(buffer)
credit_df = pd.read_parquet(buffer)

In [19]:
file_dependent = "dependent.parquet.gzip"

buffer = io.BytesIO()
s3 = boto3.resource('s3')
object_ = s3.Object('humana-data', '{}/{}'.format(prefix_rawdata_train, file_dependent))
object_.download_fileobj(buffer)
dependent_df = pd.read_parquet(buffer)

In [10]:
# condition_df = condition_df.set_index("person_id_syn")
condition_df.columns = condition_df.columns.str.lower()

condition_df[feature_column_cols['condition']['categorical_cols']] = np.where(condition_df[feature_column_cols['condition']['categorical_cols']] != 0, 1,0)
condition_df = sum_feature_generation(df = condition_df, categories = condition_categories)
condition_fe_df = total_sum_features(df = condition_df)




In [11]:
num_cols = feature_column_cols['condition']['numerical_cols'] + feature_column_cols['credit']['numerical_cols'] + feature_curated_cols['condition']['numerical_cols']  + feature_curated_cols['credit']['numerical_cols'] 
num_dtypes = {x:"float64" for x in num_cols}

In [12]:
cat_cols = feature_column_cols['condition']['categorical_cols'] + feature_curated_cols['condition']['categorical_cols'] 
cat_dtypes = {x:"category" for x in cat_cols}

In [13]:
feature_dtypes = {**num_dtypes, **cat_dtypes}

In [18]:
df_fe = credit_fe_df.merge(condition_fe_df, how='left', left_index=True, right_index=True)

In [28]:
# df_fe = dependent_df.merge(df_fe, how ='left', left_index=True, right_index=True)

In [25]:
# bucket = 'my_bucket_name' # already created on S3
from io import StringIO # python3; python2: BytesIO 
csv_buffer = StringIO()
df_fe.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
prefix_intermediate  = 'intermediate/data'
filename_fe = "train_fe.csv"

fe_path = prefix_intermediate + '/' + filename_fe
s3_resource.Object(bucket, fe_path).put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '11K2FR9ZG2051NM0',
  'HostId': 'a54XhEZPcEz6nGXiedSNTNOPAF/YaVCKqofoWACLop13E51MPT0glJWUarZWGIFQ8pj/ZHOSgt0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'a54XhEZPcEz6nGXiedSNTNOPAF/YaVCKqofoWACLop13E51MPT0glJWUarZWGIFQ8pj/ZHOSgt0=',
   'x-amz-request-id': '11K2FR9ZG2051NM0',
   'date': 'Wed, 16 Nov 2022 16:21:29 GMT',
   'x-amz-version-id': '7tpLP6tbaUIX9klOXREEyP5vt8vFkfz1',
   'etag': '"8beaf1b0eed74d9e5e739524b237635a"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"8beaf1b0eed74d9e5e739524b237635a"',
 'VersionId': '7tpLP6tbaUIX9klOXREEyP5vt8vFkfz1'}